In [14]:
import os
import sys
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from rdkit import Chem, RDLogger
import sqlite3
from tqdm import tqdm

# Suppress RDKit warnings for cleaner output during generation checks
RDLogger.DisableLog('rdApp.*')

# =========================================
# CONFIGURATION (SIMPLIFIED & TUNED)
# =========================================
class Config:
    # --- System & Data Paths ---
    CHEMPL_DB_PATH = 'DL_ENDSEM__DATASET/chembl_35/chembl_35_sqlite/chembl_35.db'
    MODEL_DIR = 'models_simplified_v2' # New dir for v2 run

    # --- Model Hyperparameters (SCALED DOWN) ---
    MAX_ATOMS = 9
    ATOM_LIST = ['C', 'N', 'O', 'F', 'S', 'Cl'] 
    NUM_ATOM_TYPES = len(ATOM_LIST) + 1 
    
    BOND_TYPES = [Chem.rdchem.BondType.ZERO,
                  Chem.rdchem.BondType.SINGLE,
                  Chem.rdchem.BondType.DOUBLE,
                  Chem.rdchem.BondType.TRIPLE,
                  Chem.rdchem.BondType.AROMATIC]
    NUM_BOND_TYPES = len(BOND_TYPES)

    LATENT_DIM = 128
    BATCH_SIZE = 64
    LR_G = 1e-4
    LR_D = 1e-4
    LAMBDA_GP = 10
    N_CRITIC = 5
    NUM_EPOCHS = 500   # INCREASED TO 100
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SAVE_EVERY = 10    # Save less frequently for long run
    RESUME_EPOCH = 100

print(f"Running on device: {Config.DEVICE}")

# =========================================
# DATA UTILS
# =========================================
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return None
    num_atoms = mol.GetNumAtoms()
    if num_atoms > Config.MAX_ATOMS or num_atoms < 2: return None

    X = np.zeros((Config.MAX_ATOMS, Config.NUM_ATOM_TYPES), dtype=np.float32)
    for i, atom in enumerate(mol.GetAtoms()):
        sym = atom.GetSymbol()
        if sym in Config.ATOM_LIST: X[i, Config.ATOM_LIST.index(sym)] = 1.0
        else: return None 
    for i in range(num_atoms, Config.MAX_ATOMS): X[i, -1] = 1.0

    A = np.zeros((Config.MAX_ATOMS, Config.MAX_ATOMS, Config.NUM_BOND_TYPES), dtype=np.float32)
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        b_type = bond.GetBondType()
        try:
            ch_idx = Config.BOND_TYPES.index(b_type)
            A[i, j, ch_idx] = A[j, i, ch_idx] = 1.0
        except ValueError: continue
    for i in range(Config.MAX_ATOMS):
        for j in range(Config.MAX_ATOMS):
            if np.sum(A[i,j,:]) == 0: A[i,j,0] = 1.0
    return X, A

def graphs_to_mols(node_X, adj_A, hard=True):
    mols = []
    if hard:
        if isinstance(node_X, torch.Tensor): node_X = torch.argmax(node_X, dim=-1).detach().cpu().numpy()
        if isinstance(adj_A, torch.Tensor): adj_A = torch.argmax(adj_A, dim=-1).detach().cpu().numpy()

    for b in range(node_X.shape[0]):
        mol = Chem.RWMol()
        atom_indices = []
        for i in range(Config.MAX_ATOMS):
            atom_type = node_X[b, i]
            if atom_type == len(Config.ATOM_LIST): continue
            atom_indices.append(mol.AddAtom(Chem.Atom(Config.ATOM_LIST[atom_type])))
        for i in range(len(atom_indices)):
            for j in range(i + 1, len(atom_indices)):
                bond_idx = adj_A[b, i, j]
                if bond_idx != 0:
                    try: mol.AddBond(atom_indices[i], atom_indices[j], Config.BOND_TYPES[bond_idx])
                    except: pass
        try:
            mol = mol.GetMol()
            Chem.SanitizeMol(mol)
            mols.append(mol)
        except: mols.append(None)
    return mols

# =========================================
# DATASET
# =========================================
class MoleculeDataset(Dataset):
    def __init__(self):
        self.data = []
        smiles_list = []
        if os.path.exists(Config.CHEMPL_DB_PATH):
            print(f"Connecting to DB at {Config.CHEMPL_DB_PATH}...")
            try:
                conn = sqlite3.connect(Config.CHEMPL_DB_PATH)
                cursor = conn.cursor()
                print("Querying for small molecule candidates...")
                cursor.execute("SELECT canonical_smiles FROM compound_structures WHERE length(canonical_smiles) < 50 LIMIT 200000")
                smiles_list = [row[0] for row in cursor.fetchall()]
                conn.close()
            except Exception as e: print(f"DB Error: {e}")

        print(f"Filtering for molecules with {Config.MAX_ATOMS} atoms or less...")
        np.random.shuffle(smiles_list)
        for s in tqdm(smiles_list):
            res = smiles_to_graph(s)
            if res is not None: self.data.append(res)
            if len(self.data) >= 20000: break 
        print(f"Training on {len(self.data)} valid small graphs.")

    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        X, A = self.data[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(A, dtype=torch.float32)

# =========================================
# MODELS
# =========================================
class GraphTransformerLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, adj_channels):
        super().__init__()
        self.embed_dim, self.num_heads, self.head_dim = embed_dim, num_heads, embed_dim // num_heads
        self.q_proj, self.k_proj, self.v_proj = nn.Linear(embed_dim, embed_dim), nn.Linear(embed_dim, embed_dim), nn.Linear(embed_dim, embed_dim)
        self.out_proj, self.adj_proj = nn.Linear(embed_dim, embed_dim), nn.Linear(adj_channels, num_heads)
    def forward(self, x, adj):
        B, N, _ = x.shape
        Q, K, V = [proj(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2) for proj in (self.q_proj, self.k_proj, self.v_proj)]
        attn = (Q @ K.transpose(-2, -1)) / np.sqrt(self.head_dim) + self.adj_proj(adj).permute(0, 3, 1, 2)
        return self.out_proj((F.softmax(attn, dim=-1) @ V).transpose(1, 2).reshape(B, N, self.embed_dim)) + x

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(Config.LATENT_DIM, 128)
        self.fc_nodes = nn.Linear(128, Config.MAX_ATOMS * Config.NUM_ATOM_TYPES)
        self.fc_adj = nn.Linear(128, Config.MAX_ATOMS * Config.MAX_ATOMS * Config.NUM_BOND_TYPES)
        # Simpler initialization
        for m in self.modules():
            if isinstance(m, nn.Linear): nn.init.normal_(m.weight, 0, 0.02)

    def forward(self, z, temperature=1.0): # Added temperature
        h = F.leaky_relu(self.fc(z), 0.2)
        nodes = F.gumbel_softmax(self.fc_nodes(h).view(-1, Config.MAX_ATOMS, Config.NUM_ATOM_TYPES), tau=temperature, hard=False, dim=-1)
        adj = self.fc_adj(h).view(-1, Config.MAX_ATOMS, Config.MAX_ATOMS, Config.NUM_BOND_TYPES)
        return nodes, F.gumbel_softmax((adj + adj.permute(0, 2, 1, 3)) / 2.0, tau=temperature, hard=False, dim=-1)

class Critic(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Linear(Config.NUM_ATOM_TYPES, 64)
        self.gt1, self.gt2 = GraphTransformerLayer(64, 4, Config.NUM_BOND_TYPES), GraphTransformerLayer(64, 4, Config.NUM_BOND_TYPES)
        self.out = nn.Sequential(nn.Linear(64, 64), nn.LeakyReLU(0.2), nn.Linear(64, 1))
    def forward(self, x, adj):
        return self.out(torch.mean(self.gt2(self.gt1(self.embed(x), adj), adj), dim=1))

# =========================================
# TRAINING
# =========================================
def compute_gp(D, r_n, r_a, f_n, f_a):
    alpha = torch.rand(r_n.size(0), 1, 1, 1, device=Config.DEVICE)
    inter_n = (alpha.squeeze(-1) * r_n + (1 - alpha.squeeze(-1)) * f_n).requires_grad_(True)
    inter_a = (alpha * r_a + (1 - alpha) * f_a).requires_grad_(True)
    d_inter = D(inter_n, inter_a)
    grads = torch.autograd.grad(d_inter, [inter_n, inter_a], torch.ones_like(d_inter), create_graph=True, retain_graph=True)
    return ((torch.cat([g.reshape(r_n.size(0), -1) for g in grads], dim=1).norm(2, dim=1) - 1) ** 2).mean()

def train():
    os.makedirs(Config.MODEL_DIR, exist_ok=True)
    dataset = MoleculeDataset()
    if len(dataset) < Config.BATCH_SIZE: return

    loader = DataLoader(dataset, batch_size=Config.BATCH_SIZE, shuffle=True, drop_last=True)
    gen, crit = Generator().to(Config.DEVICE), Critic().to(Config.DEVICE)
    opt_G, opt_D = optim.Adam(gen.parameters(), lr=Config.LR_G, betas=(0., 0.9)), optim.Adam(crit.parameters(), lr=Config.LR_D, betas=(0., 0.9))

    start_epoch = 0
    if Config.RESUME_EPOCH and load_checkpoint(Config.RESUME_EPOCH, gen, crit): start_epoch = Config.RESUME_EPOCH

    for epoch in range(start_epoch, Config.NUM_EPOCHS):
        # Temperature annealing: 2.0 -> 0.5 over 100 epochs
        temp = max(0.5, 2.0 - 1.5 * (epoch / Config.NUM_EPOCHS))
        
        pbar = tqdm(loader, desc=f"Ep {epoch+1}/{Config.NUM_EPOCHS} [T={temp:.2f}]")
        d_loss_ep, g_loss_ep, batches = 0., 0., 0
        for r_n, r_a in pbar:
            r_n, r_a = r_n.to(Config.DEVICE), r_a.to(Config.DEVICE)
            batches += 1
            
            # Add noise to real data for stability
            r_n_noisy = r_n + 0.05 * torch.randn_like(r_n)
            r_a_noisy = r_a + 0.05 * torch.randn_like(r_a)

            for _ in range(Config.N_CRITIC):
                crit.zero_grad()
                f_n, f_a = gen(torch.randn(r_n.size(0), Config.LATENT_DIM, device=Config.DEVICE), temperature=temp)
                d_loss = crit(f_n.detach(), f_a.detach()).mean() - crit(r_n_noisy, r_a_noisy).mean() + Config.LAMBDA_GP * compute_gp(crit, r_n, r_a, f_n.detach(), f_a.detach())
                d_loss.backward()
                opt_D.step()
            
            gen.zero_grad()
            g_loss = -crit(*gen(torch.randn(r_n.size(0), Config.LATENT_DIM, device=Config.DEVICE), temperature=temp)).mean()
            g_loss.backward()
            opt_G.step()
            d_loss_ep += d_loss.item(); g_loss_ep += g_loss.item()
            pbar.set_postfix({'D': d_loss.item(), 'G': g_loss.item()})

        if (epoch+1) % Config.SAVE_EVERY == 0:
            torch.save(gen.state_dict(), f"{Config.MODEL_DIR}/g_{epoch+1}.pth")
            torch.save(crit.state_dict(), f"{Config.MODEL_DIR}/c_{epoch+1}.pth")
            evaluate(epoch+1, gen)

def load_checkpoint(epoch, gen, crit=None):
    g_path = os.path.join(Config.MODEL_DIR, f'g_{epoch}.pth')
    if not os.path.exists(g_path): return False
    gen.load_state_dict(torch.load(g_path, map_location=Config.DEVICE))
    if crit: crit.load_state_dict(torch.load(os.path.join(Config.MODEL_DIR, f'c_{epoch}.pth'), map_location=Config.DEVICE))
    return True

def evaluate(epoch, gen=None):
    if gen is None:
        gen = Generator().to(Config.DEVICE)
        load_checkpoint(epoch, gen)
    gen.eval()
    valid_mols = []
    with torch.no_grad():
        for _ in range(5):
             mols = graphs_to_mols(*gen(torch.randn(100, Config.LATENT_DIM, device=Config.DEVICE), temperature=0.1)) # Low temp for eval
             valid_mols.extend([Chem.MolToSmiles(m) for m in mols if m])
    unique = set(valid_mols)
    print(f"\nEp {epoch} Results (N=500) | Valid: {(len(valid_mols)/500)*100:.1f}% | Unique: {len(unique)}")
    if unique: print(f"Samples: {list(unique)[:3]}")
    print("-" * 50)
    gen.train()

if __name__ == '__main__':
    if len(sys.argv) > 2 and sys.argv[1] == 'eval': evaluate(int(sys.argv[2]))
    else: train()

Running on device: cuda
Connecting to DB at DL_ENDSEM__DATASET/chembl_35/chembl_35_sqlite/chembl_35.db...
Querying for small molecule candidates...
Filtering for molecules with 9 atoms or less...


100%|██████████| 200000/200000 [00:26<00:00, 7457.48it/s]


Training on 1431 valid small graphs.


Ep 110/500 [T=1.67]: 100%|██████████| 22/22 [00:03<00:00,  6.94it/s, D=-0.89, G=-0.736] 



Ep 110 Results (N=500) | Valid: 10.8% | Unique: 32
Samples: ['C=C(C)C=C=O', 'C.C=O.CC1CC1', 'CC1(C)C=N1.CO']
--------------------------------------------------


...


Ep 360 Results (N=500) | Valid: 0.0% | Unique: 0
--------------------------------------------------


Ep 370/500 [T=0.89]: 100%|██████████| 22/22 [00:01<00:00, 11.36it/s, D=-929, G=31.3]    



Ep 370 Results (N=500) | Valid: 0.0% | Unique: 0
--------------------------------------------------


Ep 373/500 [T=0.88]:  14%|█▎        | 3/22 [00:00<00:01, 11.14it/s, D=-984, G=31.4]    


KeyboardInterrupt: 

In [15]:
evaluate(250)


Ep 250 Results (N=500) | Valid: 59.8% | Unique: 109
Samples: ['C.C1=C2OC12.C=CC', 'C.CCC.CO', 'C=C1C(C)N1CC1C=C1']
--------------------------------------------------


In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# --- Configuration (Make sure this matches your training script) ---
class Config:
    MODEL_DIR = 'models_simplified_v2' 
# ---

def plot_training_history():
    log_path = os.path.join(Config.MODEL_DIR, 'training_log.csv')
    if not os.path.exists(log_path):
        print(f"Error: Log file not found at {log_path}")
        return
    
    try:
        df = pd.read_csv(log_path)
        # Calculate rolling averages for smoother plots
        # Adjust window size if you have more/less data; 1% of total iterations is a good start
        window_size = max(10, len(df) // 100)
        df['D_rolling'] = df['D_loss'].rolling(window=window_size).mean()
        df['G_rolling'] = df['G_loss'].rolling(window=window_size).mean()

        plt.figure(figsize=(12, 5))
        
        # --- Discriminator Plot ---
        plt.subplot(1, 2, 1)
        plt.plot(df['D_loss'], alpha=0.2, label='Raw D Loss', color='blue')
        plt.plot(df['D_rolling'], label=f'Smoothed D Loss (win={window_size})', color='navy')
        plt.title('Discriminator Loss')
        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.5)

        # --- Generator Plot ---
        plt.subplot(1, 2, 2)
        plt.plot(df['G_loss'], alpha=0.2, label='Raw G Loss', color='orange')
        plt.plot(df['G_rolling'], label=f'Smoothed G Loss (win={window_size})', color='red')
        plt.title('Generator Loss')
        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.5)

        plt.tight_layout()
        save_path = os.path.join(Config.MODEL_DIR, 'training_history.png')
        plt.savefig(save_path)
        plt.show() # Display the plot in the notebook
        
        print(f"Plot saved to {save_path}")
        
    except Exception as e:
        print(f"Could not plot history: {e}")

# --- Call the function ---
plot_training_history()

Error: Log file not found at models_simplified_v2\training_log.csv
